## TODO

- Seperate Application to merge all DFs
- [X] Layout with grid to eliminate all the extra icons
- fix the tools availble to the user
- Create Dataframes for each CSV found in a directory
- Selection of Location
- Selection of CSV files

- Ask advice from Naomi


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os


from bokeh.layouts import layout, column, row, gridplot
from bokeh.models import CustomJS, Slider, ColumnDataSource, WidgetBox
from bokeh.plotting import figure, show, output_notebook, output_file

#output_notebook()

<IPython.core.display.Javascript object>

['datetime',
 ' TC_4',
 ' TC_3',
 ' TC_2',
 ' TC_1',
 ' TC_8',
 ' TC_7',
 ' TC_6',
 ' TC_5',
 'Fan R',
 'Fan L',
 'T_R',
 'RH_R',
 'T_L',
 'RH_L',
 'T_Out',
 'RH_Out',
 'Unnamed: 17']

In [4]:
def calculateDewPoint(t, rh):
    '''Based on the August-Roche-Magnus approximation, considered valid for:
        0 ºC < T < 60 ºC
        1% < RH < 100%
        0 ºC < Td < 50 ºC'''
    a = 17.625
    b = 243.04
    rh = (rh/100)
    
    T_d = ( b * ( np.log(rh) + (a*t)/(b+t) )) / (a - np.log(rh) - (a*t)/(b+t))
    return T_d

In [5]:
plot_config = dict(tools="pan, xwheel_zoom, box_select, save")

def plotChamber(side, x_range, y_range, w=600, h=450):
    '''Generate Chamber Condition Plots and pass back x range for all plots'''
    x = df['datetime'].values
    
    if side == "Left":
        cols = ['T_L', 'RH_L', 'T_Out', 'RH_Out']
    else:
        cols = ['T_R', 'RH_R', 'T_Out', 'RH_Out']
    
    #calculate dew points
    dp = calculateDewPoint(df[cols[0]].values, df[cols[1]].values)
    dp_o = calculateDewPoint(df[cols[2]].values, df[cols[3]].values)
    
    p = figure(
        title="{} Chamber Conditions".format(side),
        y_axis_label='Chamber Temperature/RH, C/%',
        x_axis_type="datetime",
        height=h,
        width=w,
        **plot_config
    ) 
    
    p.line(x, df[cols[0]].values, legend="Inside Temperature", line_color="red")
    
    #p.line(x, dp, legend="Inside Dew Point", line_color="blue")
    p.line(x, df[cols[1]].values, legend="Inside RH", line_color="blue")
    
    p.line(x, df[cols[2]].values, legend="Outside Temperature", line_dash="4 4", line_width=1, line_color="red")
    #p.line(x, dp_o, legend="Outside Dew Point", line_dash="4 4", line_width=1, line_color="blue")
    p.line(x, df[cols[3]].values, legend="Outside RH", line_dash="4 4", line_width=1, line_color="blue")
        
    if x_range is None:
        x_range = p.x_range
        y_range = p.y_range
    else:
        p.x_range = x_range
        p.y_range = y_range
        
    return p, x_range, y_range
    
    
    

def plotThermocouples(side, x_range, y_range, w=600, h=150): 
    '''Generate TC plots'''
    x = df['datetime'].values
    
    if side == "Left":
        cols = [' TC_1', ' TC_2', ' TC_3', ' TC_4' ]
    else:
        cols = [' TC_5', ' TC_6', ' TC_7', ' TC_8' ]
        
    ylabel = 'Temperature, C'
    
    p1 = figure(
        #title="Heater Values",
        y_range=[0, 120],
        #x_axis_label='Time' 
        y_axis_label=ylabel,
        x_axis_type="datetime",
        x_range=x_range,
        width=w, height=h
    )
    
    if y_range is None:
        y_range = p1.y_range
    else:
        p1.y_range = y_range
        
    col = cols[0]
    
    p1.line(x, df[col].values, legend = col, color="red")
    
    col = cols[1]
    p2 = figure(x_range=x_range, x_axis_type="datetime", 
                y_axis_label=ylabel, y_range=y_range,
               width=w, height=h)
    p2.line(x, df[col].values, legend=col, line_color="red")
    
    col = cols[2]
    p3 = figure(x_range=x_range, x_axis_type="datetime", 
                y_axis_label=ylabel, y_range=y_range,
               width=w, height=h)
    p3.line(x, df[col].values, legend=col, line_color="red")
    
    col = cols[3]
    p4 = figure(x_range=x_range, x_axis_type="datetime", 
                y_axis_label=ylabel, y_range=y_range,
               width=w, height=h)
    p4.line(x, df[col].values, legend=col, line_color="red")

    return p1, p2, p3, p4, y_range

    
def plotFans(side, x_range, y_range, w=600, h=150):
    '''Generate Fan Plots'''
    x = df['datetime'].values
    
    if side == "Left":
        fan = df['Fan L'].values
    else:
        fan = df['Fan R'].values
    
    p = figure(
        y_axis_label='Fan Current, mA',
        x_axis_type="datetime", 
        x_range=x_range,
        width=w, height=h 
    )
    
    if y_range is None:
        y_range = p.y_range
    else:
        p.y_range = y_range
    
    # Define Bollinger Bands.
    # filter the noisy data and band it
    #filtered  = sm.nonparametric.lowess(fan, x,  is_sorted=True, frac=0.005, it=0)
    #upperband = filtered[:,1]
    
    upperband = fan
    lowerband = np.zeros_like(upperband)

    # Bollinger shading glyph:
    band_x = np.append(x, x[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)
    
    return p, y_range
    

In [6]:
# Generate plots
# Share the x_range between all plots
# Share the y_range between plots of similar measurement (ex temperature)
cl, xr, yr = plotChamber("Left", None, None)
cr, __, __ = plotChamber("Right", xr, yr)

fl, yr = plotFans("Left", xr, None)
fr, __ = plotFans("Right", xr, yr)

tc1, tc2, tc3, tc4, yr = plotThermocouples("Left", xr, None)
tc5, tc6, tc7, tc8, __ = plotThermocouples("Right", xr, yr)

# Save the file as an HTML
output_file('Hollow Apollo Dashboard.html')  

p = gridplot([
        [cl, cr],
        [fl, fr], 
        [tc1, tc5], 
        [tc2, tc6], 
        [tc3, tc7], 
        [tc4, tc8]], )

# Display the output in a seperate window
show(p)
